In [1]:
!pip install jiwer
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.3 MB/s eta 0:00:00


In [2]:
import os
import torch
import torchaudio
import zipfile
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import get_peft_model, LoraConfig, TaskType, PeftModel
from tqdm import tqdm

# CUDA memory config
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# Unzip audio files
zip_path = "final_segments.zip"
extract_path = "final_segments"

if not os.path.exists(extract_path):
    print("Extracting audio files...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Extraction complete.")

Extracting audio files...
Extraction complete.


In [4]:
# Dataset
class WhisperDataset(Dataset):
    def __init__(self, csv_path, processor, audio_folder, max_length=128):
        self.df = pd.read_csv(csv_path)
        self.processor = processor
        self.audio_folder = audio_folder
        self.max_length = max_length

        # Only keep rows where audio file exists
        self.df["Audio File"] = self.df["Audio File"].apply(os.path.basename)
        self.df = self.df[self.df["Audio File"].apply(
            lambda x: os.path.exists(os.path.join(self.audio_folder, x))
        )].reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        audio_filename = row["Audio File"]
        audio_path = os.path.join(self.audio_folder, audio_filename)
        text = row["Text Chunk"]

        audio_tensor, sr = torchaudio.load(audio_path)
        if sr != 16000:
            audio_tensor = torchaudio.transforms.Resample(sr, 16000)(audio_tensor)
        audio_array = audio_tensor.mean(dim=0).numpy()

        inputs = self.processor(audio_array, sampling_rate=16000, return_tensors="pt")
        labels = self.processor.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        ).input_ids
        labels[labels == self.processor.tokenizer.pad_token_id] = -100

        return {
            "input_features": inputs.input_features.squeeze(0),
            "labels": labels.squeeze(0)
        }

In [5]:
# Load Whisper model and processor
model_name = "openai/whisper-large-v3"
processor = WhisperProcessor.from_pretrained(model_name)
base_model = WhisperForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

In [6]:
# LoRA config
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"]
)

In [7]:
# Apply LoRA
model = get_peft_model(base_model, lora_config)

# Patch model for audio (input_features) instead of input_ids
class WhisperPeftWrapper(PeftModel):
    def forward(self, input_features=None, labels=None, **kwargs):
        return self.base_model(input_features=input_features, labels=labels, **kwargs)

    def generate(self, input_features=None, **kwargs):
        return self.base_model.generate(input_features=input_features, **kwargs)

adapter_config = model.peft_config["default"]
model = WhisperPeftWrapper(model.base_model, adapter_config).to(device)

# Dataset and DataLoader
dataset = WhisperDataset("aligned_final_segments.csv", processor, extract_path)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [8]:
# Training
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
model.train()

print("Starting training...\n")
for epoch in range(3):
    total_loss = 0
    pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}")
    for batch in pbar:
        input_features = batch["input_features"].to(device).half()  # cast to float16
        labels = batch["labels"].to(device)

        outputs = model(input_features=input_features, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        pbar.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1} complete - Avg Loss: {total_loss / len(dataloader):.4f}")

# Save Model
model.save_pretrained("whisper-large-peft-malay-e3")
processor.save_pretrained("whisper-large-peft-malay-e3")
print("\nModel saved to 'whisper-large-peft-malay-e3'")

Starting training...



Epoch 1: 100%|██████████| 1424/1424 [12:02<00:00,  1.97it/s, loss=0.146]


Epoch 1 complete - Avg Loss: 0.3695


Epoch 2: 100%|██████████| 1424/1424 [12:05<00:00,  1.96it/s, loss=0.396]


Epoch 2 complete - Avg Loss: 0.2250


Epoch 3: 100%|██████████| 1424/1424 [12:05<00:00,  1.96it/s, loss=0.0954]


Epoch 3 complete - Avg Loss: 0.1997

Model saved to 'whisper-large-peft-malay-e3'


In [9]:
from jiwer import wer
import pandas as pd
from tqdm import tqdm
import torch

def evaluate_model(model, processor, dataset, save_path="evaluation_results_large_e3.csv", preview_limit=100):
    model.eval()
    predictions = []
    references = []
    audio_ids = []

    # Force decoder to transcribe in Malay
    forced_decoder_ids = processor.get_decoder_prompt_ids(language="malay", task="transcribe")
    total_samples = len(dataset)

    print(f"Evaluating on {total_samples} samples...\n")

    for i in tqdm(range(total_samples), desc="Evaluating"):
        sample = dataset[i]
        input_features = sample["input_features"].unsqueeze(0).to("cuda").half()

        with torch.no_grad():
            generated_ids = model.generate(
                input_features=input_features,
                forced_decoder_ids=forced_decoder_ids,
                max_new_tokens=256,
                repetition_penalty=1.2,             # slightly stronger
                no_repeat_ngram_size=5,             # prevents loops like penyelidikan penyelidikan
                eos_token_id=processor.tokenizer.eos_token_id  # stop when <|endoftext|> is hit
            )

            transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        target = processor.tokenizer.decode(
            [t for t in sample["labels"].tolist() if t != -100],
            skip_special_tokens=True
        )

        predictions.append(transcription.strip().lower())
        references.append(target.strip().lower())
        audio_ids.append(i)

        if i < preview_limit:
            sample_wer = wer([target.strip().lower()], [transcription.strip().lower()])
            print(f"\nSample {i+1}")
            print(f"Reference:     {target}")
            print(f"Transcription: {transcription}")
            print(f"WER:           {sample_wer:.3f}\n")

    # Compute overall WER
    overall_wer = wer(references, predictions)
    print(f"\nOverall WER (Word Error Rate): {overall_wer:.3f}")

    # Save results
    df = pd.DataFrame({
        "ID": audio_ids,
        "Reference": references,
        "Prediction": predictions
    })
    df["WER"] = df.apply(lambda row: wer([row["Reference"]], [row["Prediction"]]), axis=1)
    df.to_csv(save_path, index=False)
    print(f"Results (with per-row WER) saved to: {save_path}")

    return overall_wer

In [10]:
# Evaluate on all samples from the training dataset
evaluate_model(model, processor, dataset, save_path="evaluation_results_large_e3.csv", preview_limit=100)

Evaluating on 2848 samples...



Evaluating:   0%|          | 0/2848 [00:00<?, ?it/s]Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Evaluating:   0%|          | 1/2848 [00:06<5:03:03,  6.39s/it]


Sample 1
Reference:     Pendahuluan,  pendapat  awal  saya  mengenai  masalah  yang  sedang  dihadapi  oleh  orang  melayu  dibentangkan  untuk  pertama  kalinya  sewaktu  menjawab  cabaran  yang  dikemukakan  oleh  Profesor  Ungku  Aziz,  Professor  Ekonomi,  (sekarang  Professor  Diraja  dan  Naib
Transcription:  Pendahuluan,  pendapat  awal  saya  mengenai  masalah  yang  sedang  dihadapi  oleh  orang  melayu  dibentankan  untuk  pertama  kalinya  suatu  menjawab  cabaran  yang  dikemukakan  oleh  Profesor  Ungku  Aziz,  Profesor ekonomi,  sekarang  profesor  diraja  dan  naik.
WER:           0.188



Evaluating:   0%|          | 2/2848 [00:12<5:00:46,  6.34s/it]


Sample 2
Reference:     Canselor  di  Universiti  Malaya.  Di  dalam  satu  seminar  yang  diadakan  di  Kuala  Lumpur  pada  tahun  1966,  ketika  membincangkan  sebab-sebab  lemahnya  pencapaian  pelajar-pelajar  melayu,  saya  telah  menyebut,  antara  lain,
Transcription:  chanceler  di  universiti  Malaya.  Di  dalam  satu  seminar  yang  diadakan  dikuala  lumpur  pada  tahun  1966,  ketika  membincangkan  sebab-sebap  lemahnya  pencapaian  pelajar  peraja  melayu,  saya  telah  menyebut  antara  lain,  dan  terlalu  berjagaan  keadaannya.
WER:           0.393



...

Evaluating:   4%|▎         | 100/2848 [13:59<9:28:33, 12.41s/it]


Sample 100
Reference:     Kauem -kauem  sekutu  dalam  kerja  antperikatan  sedah  akan  senketasin  ketemasa  lalu  dan  amat  berhati -hati  agar  tidak  membangkitkan  masalah  perkawan.
Transcription: Kaum  kaum  sekutu  dalam  kerajaan  perikatan  sedah,  akan  senketasengketer  masa  lalu  dan  amat  berhati-hatih  agar  tidak  membangkitkan  masalah  perkawam.  Kaujurnya  diberi  kebuncahan  tersebut,  yang  menyebabkan  penyakit  kerana  mereka  mengalirkan  bahawa  dia  ada  benda  untuk  muka  perekaian  atau  makna  jenis  dengan  syabuk  sehingga  kita  tak  boleh  melayui  apa  ini?  Dan  saya  ingin  menyanyikan  baginda  pada  selama  itu  adalah  pengguna  yang  merata  secara  serta  pekerjaannya  dan  semangkat  seseorang  sudah  terserbaiki  kepada  pertumbuhkan  sama  suatu  seperti  setidakan  persudahan  kesempatnanya  bers
WER:           3.400



Evaluating: 100%|██████████| 2848/2848 [6:32:06<00:00,  8.26s/it]



Overall WER (Word Error Rate): 1.140
Results (with per-row WER) saved to: evaluation_results_large_e3.csv


1.139607032057911

In [11]:
import shutil

shutil.make_archive("whisper-large-peft-malay-e3", 'zip', "whisper-large-peft-malay-e3")
print("Zipped model folder: whisper-large-peft-malay-e3.zip")

Zipped model folder: whisper-large-peft-malay-e3.zip


In [12]:
from google.colab import files
files.download("evaluation_results_large_e3.csv")
files.download("whisper-large-peft-malay-e3.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>